# Datasets merging

In [1]:
import pandas as pd
import string
import ast
Data_path = 'Data/'

## Load the movies data

In [2]:
info_df = pd.read_csv(Data_path+'tmdb_5000_movies.csv', parse_dates=['release_date'])
credit_df = pd.read_csv(Data_path+'tmdb_5000_credits.csv')
# create a columns with the year of the coresponding Oscar ceremony for merging latter on
info_df['ceremony_year'] = info_df['release_date'].apply(lambda date: date.year+1)
movie_df = pd.merge(info_df, credit_df.drop(columns={'title'}), left_on='id', right_on='movie_id', how='outer') \
                        .drop(columns=['movie_id'])

Let's check is there are some NAN sneeking around.

In [3]:
# check if there some nan values
pd.DataFrame(movie_df.isna().sum()).rename(columns={0:'nbr of NAN'})

,nbr of NAN
budget,0
genres,0
homepage,3091
id,0
keywords,0
original_language,0
original_title,0
overview,3
popularity,0
production_companies,0


Looks like that two movies are missing the runtime values and one seems to miss a date. Lets find on the web the duration of those two movie and complete the data. We remove the row where the date is missing as most of the values for this movie are not defined. 

In [4]:
# complete the two missing runtime from google search
idx = movie_df[movie_df.runtime.isna()].index.to_list()
durations = [98, 81] # from google search
movie_df.loc[idx, 'runtime'] = durations

In [5]:
# drop the row where the date is ill
movie_df.dropna(subset=['release_date'], inplace=True)

In [6]:
# check if all good
pd.DataFrame(movie_df.isna().sum()).rename(columns={0:'nbr of NAN'})

,nbr of NAN
budget,0
genres,0
homepage,3090
id,0
keywords,0
original_language,0
original_title,0
overview,3
popularity,0
production_companies,0


In [7]:
# set the release_year in int
movie_df.release_date = movie_df.release_date.astype(int)

## Load Oscars and Golden Globes data

In [8]:
oscar_df = pd.read_csv(Data_path+'oscars_website.csv').drop(columns='Unnamed: 0')

In [9]:
GG_df = pd.read_csv(Data_path+'goldenglobes_website.csv').drop(columns='Unnamed: 0')

The title may differ by some difference in the case and the presence of different punctuation. In order to ensure a proper merging, we format them in lower case and we remove the punctuation.

In [10]:
def parse_title(s):
    """
    Parse a string : to lowercase, then remove punctuation
    
    INPUT : 
        |---- s : [string] string to parse 
    OUTPUT 
        |---- s_parsed : [string] parsed string
    """
    s = s.lower()
    s = s.translate(str.maketrans('', '', string.punctuation))
    return s

In [11]:
movie_df.title = movie_df.title.apply(parse_title)
oscar_df.Films = oscar_df.Films.apply(parse_title)
GG_df.Films = GG_df.Films.apply(parse_title)

The scrapped Golden Globes title data is not perfectly clean as there are sometime the released year of the movie in the title. In addition, there is a space in the beginnig of the title. Finnaly the movie starting with _The_, _A_, _An_ have this determinant at the end of the title instead of in the begining.

In [12]:
def clean_GoldenGlobes_title(row):
    """
    Clean the title from GoldenGlobes data. Remove first space. 
    Move 'the', 'a', 'an' in front. Remove released year in the name.
    
    INPUT : 
        |---- row : [list] Golden globes data row [Films, Year, Nomination, GoldenGlobes] 
    OUTPUT 
        |---- s_cleaned : [string] cleaned title
    """
    s = row[0]
    year = row[1]
    # strip to remove space in begining 
    s = s.strip()
    
    # put determinant in front if necessary : Ex : Purge The --> The Purge
    word_list = s.split(' ')
    if word_list[-1] in ['the', 'a', 'an', 'la', 'le']:
        word_list.insert(0, word_list.pop()) # put the last item i first place
    
    # remove the year in title : Ex : title (1985) or title - 1985 
    if word_list[-1] in [str(year), str(year-1)]:
        word_list.pop()
    
    s = ' '.join(word_list)
    return s

In [13]:
GG_df.Films = GG_df.apply(clean_GoldenGlobes_title, axis=1)

## Merge the dataframes

Because there are multiple movies with the same name : _Titanic_ of 1997 and _Titanic_ of 1953, We merge the dataframe based on both the movie title and the ceremony year. 

In [14]:
# merge on both title and ceremony_year to differentiate two Titanics movies or two King knog movies from different years
df = pd.merge(movie_df, oscar_df, how='left', left_on=['ceremony_year', 'title'], right_on=['Year', 'Films'])
df[['Oscars','Nominations']] = df[['Oscars','Nominations']].fillna(0)

In [15]:
# merge GoldenGlobes
df = pd.merge(df, GG_df, how='left', \
                        left_on=['ceremony_year', 'title'], \
                        right_on=['Year', 'Films'], \
                        suffixes=['_Oscars', '_GoldenGlobes'])
df[['GoldenGlobes','Nominations_GoldenGlobes']] = df[['GoldenGlobes','Nominations_GoldenGlobes']].fillna(0)

In [16]:
df.drop(columns=['Films_Oscars', 'Year_Oscars', 'Films_GoldenGlobes', 'Year_GoldenGlobes'], inplace=True)

Get the total awards and nominations

In [17]:
df['Nominations'] = df.loc[:,['Nominations_GoldenGlobes', 'Nominations_Oscars']].sum(axis=1)
df['Awards'] = df.loc[:,['GoldenGlobes', 'Oscars']].sum(axis=1)

Then we save the merged and cleaned dataframes

In [18]:
# save merged data
df.to_csv(Data_path+'merged_data.csv')

## Build features dataframe 

In [19]:
def get_list(df, col, min_nbr=0, key='name'):
    """
    Get the number of unique values in the dictionnaries of the columns col from the data. 
    INPUT
        |---- data [pandas Dataframe] the dataframe with the data 
        |---- col [string array] column name to count unique values
        |---- min_nbr [int] the minimum number of apperance of the value to be kepts
    OUTPUT 
        |---- l [list] list of unique values 
    """
    tmp = df.copy()
    tmp[col] = tmp[col].apply(lambda x : [value[key] for value in ast.literal_eval(x)])
    tmp = tmp[['title',col]].explode(col)
    tmp = tmp[col].value_counts()
    return list(tmp[tmp > min_nbr].index)

def add_dummy_features(df, col, min_nbr, key='name'):
    """ 
    Convert the columns col from the dataframe df as dummy variable 
    for each word appearing more that min_nbr.
    INPUT
        |---- df [pandas Dataframe] the dataframe with the data 
        |---- col [string array] column name to dummify
        |---- min_nbr [int] the minimum number of apperance of the value to be kepts
    OUTPUT 
        |---- df [pandas Dataframe] the dataframe with the data dummified
    """
    # get the list of possible value in col
    val_list = get_list(df, col, min_nbr=min_nbr, key=key)
    # keep only the value in val_list
    X = df[col].apply(lambda x : [value[key] for value in ast.literal_eval(x)])
    X = X.apply(lambda x : [val for val in x if val in val_list])
    # get the list as dummy variable
    tmp = pd.get_dummies(X.apply(pd.Series), prefix='', prefix_sep='').sum(level=0, axis=1)
    # add the new feature to the dataframe
    return pd.concat([df, tmp], axis=1).drop(columns=[col])

In [20]:
features_df = df[['title','budget', 'genres', \
                            'popularity', 'revenue','runtime','vote_average','vote_count']]

# add genres as dummy features (for exploration and learning)
features_df = add_dummy_features(features_df, 'genres', min_nbr=0)

features_df.to_csv(Data_path+'features.csv')

In [21]:
label_df = df[['title', 'GoldenGlobes', 'Nominations_GoldenGlobes', 'Oscars', \
                       'Nominations_Oscars', 'Nominations', 'Awards']].copy()
# save 
label_df.to_csv(Data_path+'labels.csv')